In [ ]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from PIL import Image
import cripser as cr
from gudhi.representations import Landscape
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import UnidentifiedImageError
import seaborn as sns
from collections import Counter
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

https://github.com/luangtatipsy/intel-image-classification/blob/master/101_result_analysis.ipynb

In [ ]:
def dataloader(directory):
    records = []
    for class_path in sorted([d for d in directory.iterdir() if d.is_dir()]):
        for image_path in class_path.glob("*.jpg"):
            records.append({"image": str(image_path), "class": class_path.name})
    return pd.DataFrame(records)

In [ ]:
train_dir = Path("../data/seg_train/seg_train")
train_df = dataloader(train_dir)

In [ ]:
fig, axes = plt.subplots(2, 3, figsize = (12, 8))
fig.suptitle("Intel Scene Classification Original Images", fontsize=16)

for ax, class_name in zip(axes.flat, sorted(train_df["class"].unique())[:6]):
    img_path = train_df[train_df["class"] == class_name].iloc[0]["image"]
    img = Image.open(img_path)
    ax.imshow(img)
    ax.set_title(class_name)
    ax.axis("off")

plt.tight_layout()
plt.show()

### Data Exploration

In [ ]:
def check_corrupt_images(df):
    corrupt_paths = []
    for path in df["image"]:
        try:
            with Image.open(path) as img:
                img.verify()
        except (UnidentifiedImageError, IOError, OSError):
            corrupt_paths.append(path)
    return corrupt_paths

corrupt_imgs = check_corrupt_images(train_df)
print("Corrupt images:", corrupt_imgs)

In [ ]:
sns.countplot(data=train_df, x="class")
plt.title("Class Distribution")
plt.xticks(rotation=45)
plt.show()

In [ ]:
image_stds = []

for idx, row in train_df.iterrows():
    img = Image.open(row["image"]).convert("L")  # Convert to grayscale for std
    std_val = np.std(np.array(img))
    image_stds.append(std_val)

train_df["std"] = image_stds

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(train_df["std"], bins=50, kde=True)
plt.title("Image Standard Deviation Distribution")
plt.xlabel("Pixel Intensity Standard Deviation")
plt.ylabel("Count")
plt.grid(True)
plt.show()

In [ ]:
lowest_std_df = train_df.sort_values("std").head(10)

fig, axes = plt.subplots(2, 5, figsize=(15, 6))
for ax, (_, row) in zip(axes.flat, lowest_std_df.iterrows()):
    img = Image.open(row["image"])
    ax.imshow(img)
    ax.set_title(f"std: {row['std']:.1f}")
    ax.axis("off")

plt.suptitle("10 Most Uniform Images (Lowest Std)", fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.88)
plt.show()

In [ ]:
train_df["dimensions"] = train_df["image"].apply(lambda x: f"{Image.open(x).size[0]}x{Image.open(x).size[1]}")

dimension_counts = train_df["dimensions"].value_counts().reset_index()
dimension_counts.columns = ["dimensions", "count"]

plt.figure(figsize=(12, 6))
sns.barplot(data=dimension_counts, x="dimensions", y="count")
plt.title("Image Dimension Distribution (Exact WxH)")
plt.xlabel("Dimensions")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# First, parse the image size directly
train_df["size"] = train_df["image"].apply(lambda x: Image.open(x).size)

# Filter only 150x150
train_df = train_df[train_df["size"] == (150, 150)].reset_index(drop=True)

print(f"Filtered dataset size: {train_df.shape}")

In [ ]:
for sample_class in ["buildings", "forest", "glacier", "mountain", "sea", "street"]:
    fig, axes = plt.subplots(1, 5, figsize=(15, 3))
    fig.set_dpi(300)
    fig.patch.set_alpha(0)  # Make the figure background transparent
    
    sample_paths = train_df[train_df["class"] == sample_class]["image"].sample(5)
    for ax, img_path in zip(axes, sample_paths):
        img = Image.open(img_path)
        ax.imshow(img)
        ax.axis("off")
    
    class_title = sample_class.capitalize()
    plt.suptitle(class_title, fontsize=24, fontweight='bold', fontname='Times New Roman', color='black')
    plt.show()

# Train / Test Split

In [ ]:
from sklearn.model_selection import train_test_split

train_list, val_list = [], []

for class_name, group in train_df.groupby("class"):
    train_split, val_split = train_test_split(
        group, test_size=0.2, random_state=42, shuffle=True, stratify=None
    )
    train_list.append(train_split)
    val_list.append(val_split)

train_split_df = pd.concat(train_list).reset_index(drop=True)
val_split_df = pd.concat(val_list).reset_index(drop=True)

print(f"Train split: {train_split_df.shape}, Val split: {val_split_df.shape}")

https://github.com/shizuo-kaji/CubicalRipser_3dim/blob/master/demo/cubicalripser.ipynb

In [ ]:
def compute_persistence_diagrams(img2d):
    pd_v = cr.computePH(img2d)
    pd_v = pd_v[:, :3]
    return [pd_v[pd_v[:, 0] == i][:, 1:] for i in range(3)]

In [ ]:
def diagram_to_landscape(diag, num_landscapes = 3, resolution = 100):
    L = Landscape(num_landscapes = num_landscapes, resolution = resolution)
    return L.fit_transform([diag])[0]

In [ ]:
def compute_image_topology_vector(img_path, dims = [1], num_landscapes = 3, resolution = 100):
    img2d = np.array(Image.open(img_path).convert("L"))
    pds_v = compute_persistence_diagrams(img2d)
    vecs = [diagram_to_landscape(pds_v[dim], num_landscapes, resolution) for dim in dims]
    return np.concatenate(vecs)

In [ ]:
from tqdm import tqdm

train_vectors = [compute_image_topology_vector(row["image"], dims=[1], num_landscapes=3, resolution=100)
                 for _, row in tqdm(train_split_df.iterrows(), total=len(train_split_df))]
train_vector_df = pd.DataFrame(train_vectors)
train_full_df = pd.concat([train_split_df, train_vector_df], axis=1)

val_vectors = [compute_image_topology_vector(row["image"], dims=[1], num_landscapes=3, resolution=100)
               for _, row in tqdm(val_split_df.iterrows(), total=len(val_split_df))]
val_vector_df = pd.DataFrame(val_vectors)
val_full_df = pd.concat([val_split_df, val_vector_df], axis=1)

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(12, 8))
fig.suptitle("Persistence Diagram of Intel Scene Classification Images", 
             fontsize=24, fontweight='bold', fontname='Times New Roman', color='black')

for ax, class_name in zip(axes.flat, sorted(train_df["class"].unique())[:6]):
    img_path = train_df[train_df["class"] == class_name].iloc[0]["image"]
    img = np.array(Image.open(img_path).convert("L"))
    pdgms = compute_persistence_diagrams(img)
    diag = pdgms[1]
    ax.scatter(diag[:, 0], diag[:, 1], s=10)
    ax.plot([0, diag[:, 1].max()], [0, diag[:, 1].max()], 'r--', linewidth=1)
    # Set subplot title in bold Times New Roman with first letter capitalized
    ax.set_title(class_name.capitalize(), 
                 fontdict={'fontsize':16, 'fontweight':'bold', 'fontname':'Times New Roman', 'color':'black'})
    ax.axis("equal")

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(12, 8))
fig.suptitle("Persistence Landscapes of Intel Scene Classification Images", 
             fontsize=24, fontweight='bold', fontname='Times New Roman', color='black')

for ax, class_name in zip(axes.flat, sorted(train_df["class"].unique())[:6]):
    img_path = train_df[train_df["class"] == class_name].iloc[0]["image"]
    vec = compute_image_topology_vector(img_path, dims=[1], num_landscapes=3, resolution=100)
    for i in range(3):
        ax.plot(vec[i * 100:(i + 1) * 100])
    ax.set_title(class_name.capitalize(), 
                 fontdict={'fontsize':16, 'fontweight':'bold', 'fontname':'Times New Roman', 'color':'black'})
    ax.axis("off")

plt.tight_layout()
plt.show()


https://github.com/navpreetnp7/Image-Classification-using-Densenet/blob/main/SynergyLabs.Task.ipynb

# DenseNet Implementation

In [ ]:
class ImageToLandscapeDataset(Dataset):
    def __init__(self, dataframe, transform = None):
        self.df = dataframe
        self.transform = transform
        self.vec_cols = [col for col in dataframe.columns if isinstance(col, int)]

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row["image"]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        vec = torch.tensor(row[self.vec_cols].values.astype(np.float32))
        return img, vec

In [ ]:
class DenseNetToLandscape(nn.Module):
    def __init__(self, out_dim, unfreeze_top_k=0):
        super().__init__()
        self.base = models.densenet121(pretrained=True)
        
        # Freeze all layers initially
        for param in self.base.parameters():
            param.requires_grad = False

        # Optionally unfreeze top-k layers of features
        if unfreeze_top_k > 0:
            layers = list(self.base.features.children())[::-1]
            count = 0
            for layer in layers:
                for param in layer.parameters():
                    param.requires_grad = True
                count += 1
                if count >= unfreeze_top_k:
                    break

        in_features = self.base.classifier.in_features
        self.base.classifier = nn.Linear(in_features, out_dim)

    def forward(self, x):
        return self.base(x)


In [ ]:
from tqdm import tqdm

def train_landscape_model(model, train_loader, val_loader, optimizer, loss_fn, device, epochs=10):
    model.to(device)
    train_losses, val_losses = [], []

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        # Wrap the train_loader in tqdm
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]", leave=False)
        for images, vectors in pbar:
            images, vectors = images.to(device), vectors.to(device)
            optimizer.zero_grad()
            preds = model(images)
            loss = loss_fn(preds, vectors)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
            pbar.set_postfix(loss=loss.item())

        avg_train_loss = train_loss / len(train_loader.dataset)
        train_losses.append(avg_train_loss)

        # Validation loop
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            pbar = tqdm(val_loader, desc=f"Epoch {epoch+1} [Val]", leave=False)
            for images, vectors in pbar:
                images, vectors = images.to(device), vectors.to(device)
                preds = model(images)
                loss = loss_fn(preds, vectors)
                val_loss += loss.item() * images.size(0)
                pbar.set_postfix(loss=loss.item())

        avg_val_loss = val_loss / len(val_loader.dataset)
        val_losses.append(avg_val_loss)

        print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}")

    return model, train_losses, val_losses


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = ImageToLandscapeDataset(train_full_df, transform=transform)
val_dataset = ImageToLandscapeDataset(val_full_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [ ]:

# Fix SHAP incompatibility by disabling in-place ReLUs
def set_relu_to_non_inplace(model):
    for module in model.modules():
        if isinstance(module, torch.nn.ReLU):
            module.inplace = False


model = DenseNetToLandscape(out_dim=300)  # 3 landscapes x 100 resolution
set_relu_to_non_inplace(model)            
optimizer = optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.MSELoss()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate_model(model, dataloader, device):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for images, vectors in dataloader:
            images, vectors = images.to(device), vectors.to(device)
            preds = model(images)
            y_true.append(vectors.cpu().numpy())
            y_pred.append(preds.cpu().numpy())

    y_true = np.vstack(y_true)
    y_pred = np.vstack(y_pred)

    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    print(f"RMSE: {rmse:.4f}")
    print(f"MAE : {mae:.4f}")
    print(f"R²  : {r2:.4f}")

    return {"RMSE": rmse, "MAE": mae, "R2": r2}

In [ ]:
import torch.optim as optim
from itertools import product
import pandas as pd

# Initialize storage for grid search results and best model tracking
results = []
best_val_loss = float('inf')
best_model = None
best_train_losses = None
best_val_losses = None
best_hyperparams = {}

# Define hyperparameter search space
lrs = [5e-4, 1e-3]
decays = [0, 1e-5]
unfreeze_k = [0, 2]  # 0 = frozen, 2 = partial

search_space = list(product(lrs, decays, unfreeze_k))

for lr, wd, k in search_space:
    print(f"\n--- Training: LR={lr}, WD={wd}, UnfreezeTopK={k} ---")
    model = DenseNetToLandscape(out_dim=300, unfreeze_top_k=k)
    # Only optimize parameters that require grad
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, weight_decay=wd)
    
    # Train for a few epochs (here 5 for speed)
    model, train_losses, val_losses = train_landscape_model(
        model, train_loader, val_loader, optimizer, loss_fn, device, epochs=10
    )
    
    # Evaluate model on validation set
    metrics = evaluate_model(model, val_loader, device)
    metrics.update({
        "LR": lr,
        "WeightDecay": wd,
        "UnfreezeTopK": k,
        "TrainLossFinal": train_losses[-1],
        "ValLossFinal": val_losses[-1],
        "TrainLosses": train_losses,
        "ValLosses": val_losses
    })
    
    results.append(metrics)
    
    # Check for best validation loss
    if val_losses[-1] < best_val_loss:
        best_val_loss = val_losses[-1]
        best_model = model  # best_model remains on GPU
        best_train_losses = train_losses
        best_val_losses = val_losses
        best_hyperparams = {"LR": lr, "WeightDecay": wd, "UnfreezeTopK": k}

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

metrics_df = pd.DataFrame(results)
summary_df = metrics_df[["LR", "WeightDecay", "UnfreezeTopK", "RMSE", "MAE", "R2", "ValLossFinal"]]
summary_df = summary_df.sort_values("ValLossFinal")

# Helper function to format numeric values: 5 significant figures and no scientific notation.
def format_val(val):
    try:
        val = float(val)
        s = f"{val:.5g}"  # 5 significant digits
        if "e" in s or "E" in s:  # Force plain decimal if scientific notation is used
            s = f"{val:.5f}"
        return s
    except:
        return str(val)

# Format each cell in summary_df (assumed to be already defined and sorted)
formatted_values = summary_df.applymap(lambda x: format_val(x)).values.tolist()

# Create a figure and axis for the table
fig, ax = plt.subplots(figsize=(10, 4))
ax.axis('off')  # Hide the axis

# Create the table using the formatted values and the DataFrame's columns as headers
table = ax.table(
    cellText=formatted_values,
    colLabels=summary_df.columns,
    cellLoc='center',
    loc='center'
)

# Set font sizes and scale the table
table.auto_set_font_size(False)
table.set_fontsize(12)
table.scale(1.2, 1.2)

# Apply styling: Bold header in Times New Roman with a light gray background
for (row, col), cell in table.get_celld().items():
    cell.set_edgecolor('black')
    if row == 0:
        cell.set_text_props(fontproperties={'weight': 'bold', 'family': 'Times New Roman'})
        cell.set_facecolor('#d3d3d3')  # Light gray for header
    else:
        cell.set_text_props(fontproperties={'family': 'Times New Roman'})

# Set a title for the table
plt.title("Grid Search Summary", fontsize=16, fontweight='bold', fontname='Times New Roman', color='black')

# Save the figure as a high-resolution image with a transparent background
plt.savefig("grid_search_summary.png", dpi=300, transparent=True, bbox_inches='tight')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
plt.plot(best_train_losses, label='Train Loss')
plt.plot(best_val_losses, label='Validation Loss')
plt.xlabel("Epoch", fontsize=14)
plt.ylabel("MSE Loss", fontsize=14)
plt.title("Training vs Validation Loss", fontsize=18, fontweight='bold', fontname='Times New Roman', color='black')
plt.legend(fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
from matplotlib.lines import Line2D
import numpy as np
from PIL import Image

colors = ["red", "green", "blue"]
class_names = sorted(val_full_df["class"].unique())

fig, axes = plt.subplots(1, 5, figsize=(20, 4))
plt.suptitle("Predicted vs True Persistence Landscapes", fontsize=24, fontweight='bold', fontname='Times New Roman', color='black')

best_model.eval()
with torch.no_grad():
    for ax, class_name in zip(axes, class_names[:5]):
        # Sample one image from this class in validation set
        sample_row = val_full_df[val_full_df["class"] == class_name].sample(1).iloc[0]
        img = Image.open(sample_row["image"]).convert("RGB")
        img_tensor = transform(img).unsqueeze(0).to(device)
        
        # True vector (assumed to be stored in columns with integer names)
        true_vec = torch.tensor(sample_row[[col for col in val_full_df.columns if isinstance(col, int)]].values.astype(np.float32))
        pred_vec = best_model(img_tensor).squeeze().cpu().numpy()
        
        xs = np.arange(100)
        for i in range(3):
            ax.plot(xs, true_vec[i * 100:(i + 1) * 100], color=colors[i], linestyle="--")
            ax.plot(xs, pred_vec[i * 100:(i + 1) * 100], color=colors[i])
        
        ax.set_title(class_name.capitalize(), fontsize=14, fontweight='bold', fontname='Times New Roman', color='black')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_frame_on(False)

# Custom legend: dashed for true, solid for predicted
custom_lines = [
    Line2D([0], [0], color="black", linestyle="--", label="True"),
    Line2D([0], [0], color="black", linestyle="-", label="Predicted")
]
fig.legend(handles=custom_lines, loc="upper right", fontsize=12)
plt.tight_layout()
plt.subplots_adjust(top=0.85, right=0.95)
plt.show()

# Feature Importance

In [ ]:
import torch.nn.functional as F
from torchvision.models import densenet121

class SHAPSafeDenseNet121(torch.nn.Module):
    def __init__(self, out_dim):
        super().__init__()
        base = densenet121(pretrained=True)

        # Freeze all layers
        for param in base.parameters():
            param.requires_grad = False

        self.features = base.features
        self.pool = torch.nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = torch.nn.Linear(base.classifier.in_features, out_dim)

        # Unfreeze final classifier layer
        for param in self.classifier.parameters():
            param.requires_grad = True

    def forward(self, x):
        features = self.features(x)
        features = F.relu(features.clone(), inplace=False)  # ✅ clone to avoid inplace error on views
        out = self.pool(features)
        out = torch.flatten(out, 1)
        out = self.classifier(out)
        return out


In [ ]:
model = SHAPSafeDenseNet121(out_dim=300)
model.to(device)
model.eval()
print()

In [ ]:
import shap
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

# ---------- SHAP Setup ----------

def setup_shap_explainer(model, background_loader, device):
    model.eval()
    model.to(device)
    background_batch = next(iter(background_loader))[0][:3].to(device)  # Use 3 background images for speed
    explainer = shap.GradientExplainer(model, background_batch)
    return explainer

# ---------- SHAP Value Computation ----------

def compute_shap_values(explainer, image_batch, top_k=1):
    """
    image_batch: Tensor of shape [B, 3, H, W], already on device.
    Returns SHAP values for top-k output dimensions.
    """
    shap_values = explainer.shap_values(image_batch, ranked_outputs=top_k)
    return shap_values  # List of length = top_k, each element is [B, 3, H, W]

# ---------- SHAP Visualization ----------

def visualize_shap_images(sample_paths, shap_values, transform, model_input_size=(3, 256, 256)):
    """
    Visualizes SHAP overlays.
      - sample_paths: List of image paths (length B)
      - shap_values: List with shape [top_k][B, 3, H, W]. Each element may be a torch.Tensor or a NumPy array.
    """
    fig, axes = plt.subplots(1, len(sample_paths), figsize=(4 * len(sample_paths), 4))
    if len(sample_paths) == 1:
        axes = [axes]  # Ensure iterable

    for i, (img_path, ax) in enumerate(zip(sample_paths, axes)):
        # Load image for display
        img = Image.open(img_path).convert("RGB")
        img_resized = img.resize((model_input_size[2], model_input_size[1]))
        img_np = np.array(img_resized) / 255.0  # Normalize for display

        # Retrieve SHAP values for this image from each output dimension
        per_output_vals = []
        for sv in shap_values:
            val = sv[i]
            if isinstance(val, torch.Tensor):
                val = val.detach().cpu().numpy()
            per_output_vals.append(val)
        
        # Check if all SHAP outputs have the same shape before stacking
        shapes = [np.array(val).shape for val in per_output_vals]
        if len(set(shapes)) == 1:
            aggregated = np.mean(np.abs(np.stack(per_output_vals)), axis=0)
        else:
            # Fallback: use the first output if shapes differ
            aggregated = np.abs(per_output_vals[0])
            
        shap_heatmap = np.mean(aggregated, axis=0)  # Average over channels → [H, W]

        ax.imshow(img_np)
        ax.imshow(shap_heatmap, cmap='hot', alpha=0.6)
        ax.axis("off")

    plt.tight_layout()
    plt.show


# ---------- Load Background + Set Up Explainer ----------

background_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
explainer = setup_shap_explainer(model, background_loader, device)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch

# For each class, sample 5 images, compute SHAP overlays in a batch, and plot them in one row.
for sample_class in ["buildings", "forest", "glacier", "mountain", "sea", "street"]:
    # Sample 5 image paths from the current class
    sample_paths = list(train_df[train_df["class"] == sample_class]["image"].sample(5))
    
    # Load and transform images into a batch
    batch_imgs = [transform(Image.open(p).convert("RGB")) for p in sample_paths]
    batch = torch.stack(batch_imgs).to(device)
    
    # Compute SHAP values for the batch (top-1 output for speed)
    shap_values = compute_shap_values(explainer, batch, top_k=1)
    
    # Create a figure with 5 subplots in one row
    fig, axes = plt.subplots(1, 5, figsize=(15, 3))
    fig.set_dpi(300)
    fig.patch.set_alpha(0)  # Transparent background
    
    # For each image in the batch, plot the original image with its SHAP heatmap overlay
    for i, ax in enumerate(axes):
        # Load the image for display (resize to a common size, e.g., 256x256)
        img = Image.open(sample_paths[i]).convert("RGB").resize((256, 256))
        img_np = np.array(img) / 255.0  # Normalize for display
        
        # Get the SHAP value for this image; since top_k=1, take the first element
        val = shap_values[0][i]
        if isinstance(val, torch.Tensor):
            val = val.detach().cpu().numpy()
        # For top_k=1, we can directly use the absolute SHAP value
        shap_overlay = np.mean(np.abs(val), axis=0)  # Average over channels to form a heatmap
        
        # Plot the original image and overlay the heatmap
        ax.imshow(img_np)
        ax.imshow(shap_overlay, cmap='hot', alpha=0.6)
        ax.axis("off")
    
    # Set a bold title in Times New Roman for the row, with first letter capitalized
    plt.suptitle(sample_class.capitalize(), fontsize=24, fontweight='bold', 
                 fontname='Times New Roman', color='black')
    plt.tight_layout()
    plt.show()
